In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from google.colab import drive

In [ ]:
# prompt: google drive link
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parent folder containing 10 class folders (adjust the path to your actual dataset)
parent_folder = '/content/drive/MyDrive/OldImage'  # Example path, change accordingly

# Output folder for augmented images
output_folder = '/content/drive/MyDrive/NewImage'  # Example path, change accordingly

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Create an ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=30,  # Rotate image by a random angle between -30 and 30 degrees
    shear_range=0.2,    # Apply random shear transformations with a factor of 0.2
    zoom_range=0.2,     # Apply random zoom in/out between 80% to 120%
    width_shift_range=0.2,  # Randomly shift the image horizontally by 20%
    height_shift_range=0.2,  # Randomly shift the image vertically by 20%
    vertical_flip=True,  # Flip the image vertically (up and down)
    horizontal_flip=True  # Flip the image horizontally (left and right)
)

# Function to apply augmentation to a single image and save the augmented images
def augment_and_save(image_path, output_folder, num_augmentations=4):  # Maximum of 4 images per original image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR (OpenCV default) to RGB

    # Reshape the image to (1, height, width, channels) for the generator
    img = img.reshape((1,) + img.shape)

    # Create the generator
    aug_iter = datagen.flow(img, batch_size=1)

    # Apply augmentation and save the images
    for i in range(num_augmentations):
        augmented_img = next(aug_iter)[0]  # Use next() instead of .next()
        augmented_img_path = os.path.join(output_folder, f"augmented_{i+1}_{os.path.basename(image_path)}")
        augmented_img = cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR)  # Convert back to BGR for saving
        cv2.imwrite(augmented_img_path, augmented_img)  # Save the augmented image

# Iterate over each class folder
for class_folder in os.listdir(parent_folder):
    class_folder_path = os.path.join(parent_folder, class_folder)

    # Ensure we are working with a directory and not a file
    if os.path.isdir(class_folder_path):
        # Create the corresponding output folder for augmented images
        class_output_folder = os.path.join(output_folder, class_folder)
        os.makedirs(class_output_folder, exist_ok=True)

        # Iterate over each image in the class folder
        for filename in os.listdir(class_folder_path):
            image_path = os.path.join(class_folder_path, filename)
            if os.path.isfile(image_path):
                augment_and_save(image_path, class_output_folder)
